In [1]:
# | echo: false
# | output: false
# | warning: false

from os import path
from typing import List, Union

import decoupler as dc
import pandas as pd
import scanpy as sc
import tomlkit

In [8]:
# | echo: true
# | output: false
# | warning: false

CLUSTERING_COL: str = "scTAB_annotation_majority_voting"
CONDIITION: Union[str, None] = None
SUBSET: Union[str, List, None] = None

In [3]:
# | echo: false
# | output: false
# | warning: false

## Pipeline parameters
with open("../config.toml", "r") as f:
    config = tomlkit.parse(f.read())

In [19]:
# | output: false
# | warning: false


# Directories
ROOT_DIR = config["basic"]["ANALYSIS_DIR"]
DIR_SAVE = path.join(ROOT_DIR, config["basic"]["DIR_SAVE"])
DIR_samples = config["basic"]["DIR_SAMPLES"]

# Basic information
ORGANISM: str = config["basic"]["ORGANISM"]
NORMALIZATION_METHOD = config["normalization"]["NORMALIZATION_METHOD"]

if CLUSTERING_COL == "":
    CLUSTERING_COL = config["clustering"]["CLUSTERING_COL"]
else:
    pass

In [5]:
adata = sc.read_h5ad(path.join(DIR_SAVE, "adata.h5ad"))

In [15]:
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)

In [20]:
adata.layers[NORMALIZATION_METHOD] = adata.X.copy()

In [32]:
sc.tl.rank_genes_groups(
    adata,
    groupby=CLUSTERING_COL,
    mask_var=SUBSET,
    layer=NORMALIZATION_METHOD,
    method="wilcoxon",
    pts=True,
    key_added="DE_WILCOX",
)

In [31]:
sc.tl.filter_rank_genes_groups(adata)